In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/nytimeshealth.txt'
dest_url = './run_data/nytimes_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

scripts.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  import pandas as pd


,tweetid,date,title
0,548662191340421120,Sat Dec 27 02:10:34 +0000 2014,Risks in Using Social Media to Spot Signs of M...
1,548579831169163265,Fri Dec 26 20:43:18 +0000 2014,RT @paula_span: The most effective nationwide ...
2,548579045269852161,Fri Dec 26 20:40:11 +0000 2014,The New Old Age Blog: Diabetes Prevention That...
3,548444679529041920,Fri Dec 26 11:46:15 +0000 2014,Well: Comfort Casseroles for Winter Dinners ht...
4,548311901227474944,Fri Dec 26 02:58:39 +0000 2014,High-Level Knowledge Before Veterans Affairs S...


In [3]:
# -------------------------------------------------
frame.shape

(6045, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Risks in Using Social Media to Spot Signs of Mental Distress http://nyti.ms/1rqi9I1'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://nyti.ms/1rqi9I1
http://newoldage.blogs.nytimes.com/2014/12/26/diabetes-prevention-that-works/
http://nyti.ms/1xm7fTi
http://nyti.ms/1xTNoO0
http://nyti.ms/13yCpvS
http://nyti.ms/13yAtUf
http://nyti.ms/1vtPNcm
http://nyti.ms/1wLtAen
http://bit.ly/1ziCflj
http://nyti.ms/1xNzyg4
http://nyti.ms/1vpIjqE
http://nyti.ms/16RCgpE
http://nyti.ms/1ziIYf6
http://nyti.ms/1CDkLqc
http://nyti.ms/1B2ThGD
http://nyti.ms/1B5PO9Q
http://www.nytimes.com/2014/12/24/science/mysterious-virus-that-killed-a-farmer-in-kansas-is-identified.html?smid
http://nyti.ms/1Hz7Pj2
http://nyti.ms/1ta9nPw
http://nyti.ms/1vaJ1ad
http://nyti.ms/1vaHQYn
http://nyti.ms/1ta9nPu
http://nyti.ms/13uMfPo
http://nyti.ms/1wDB6Yq
http://nyti.ms/1whtwhd
http://nyti.ms/1AZ0qaL
http://nyti.ms/13ukGpr
http://nyti.ms/1A1pDmz
http://pbs.twimg.com/media/B5kdIQXCQAAtcgA.jpg
http://nyti.ms/16NAVjH
http://nyti.ms/1A12TTG
http://nyti.ms/13B6Amt
http://nyti.ms/1vhYN4g
http://nyti.ms/1ATCm9m
http://nyti.ms/1xcoPMu
http://nyti.ms/1ATxhOj
htt

http://nyti.ms/1t9ZBI4
http://nyti.ms/1t9ZBI4
http://nyti.ms/1t9sXWU
http://t.co/n68F
http://nyti.ms/1AeNT1g
http://nyti.ms/1rbkJeD
http://nyti.ms/1rC44oy
http://nyti.ms/1rbkJeB
http://nyti.ms/1rC4486
http://nyti.ms/WjKwch
http://nyti.ms/YfBAXc
http://nyti.ms/1puqnXL
http://nyti.ms/1CvfG1r
http://nyti.ms/1CveJ9o
http://nyti.ms/1CvcK4X
http://nyti.ms/1r96N4M
http://nyti.ms/1qrtjJY
http://nyti.ms/1r8Uw0b
http://nyti.ms/1unN7yr
http://www.nytimes.com/2014/09/04/world/africa/cuts-at-who-hurt-response-to-ebola-crisis.html?
http://nyti.ms/1qxwGOm
http://www.cdc.gov/obesity/data/prevalence-maps.html
http://nyti.ms/1CujCj8
http://nyti.ms/1pN4Vn2
http://nyti.ms/Z7LMRG
http://nyti.ms/1rszJIS
http://nyti.ms/1oFATeE
http://nyti.ms/1rszMo0
http://nyti.ms/1CmBZGv
http://nyti.ms/1tumVEn
http://nyti.ms/1t1ch3N
http://nyti.ms/1w7v5Re
http://nyti.ms/1nxgsBi
http://nyti.ms/1qd41iY
http://nyti.ms/1nxgsBg
http://nyti.ms/1CqV22w
http://nyti.ms/1qcVegR
http://nyti.ms/1CqUAkS
http://nyti.ms/1CqU3iH
http://nyt

http://nyti.ms/1lPqj5N
http://nyti.ms/1lS6wCw
http://nyti.ms/1l6d2s8
http://nyti.ms/1mifPPk
http://nyti.ms/Nw8vjP
http://nyti.ms/1owUu3a
http://nyti.ms/1eSy3Ps
http://nyti.ms/1mhbU5i
http://nyti.ms/1gNmtpl
http://nyti.ms/1dbUGn4
http://nyti.ms/1ormncU
http://nyti.ms/1ifg61m
http://nyti.ms/1ifb5pC
http://nyti.ms/1p55H9E
http://nyti.ms/1d7LekN
http://nyti.ms/1d7NGI3
http://nyti.ms/1p4YNRB
http://nyti.ms/1d7LekJ
http://nyti.ms/1ojn2gw
http://nyti.ms/1p4sthK
http://t.co
http://nyti.ms/1fD6j0W
http://nyti.ms/OoHXSG
http://nyti.ms/1nxWQSn
http://nyti.ms/1cOT0zx
http://newoldage.blogs.nytimes.com/2014/03/14/mammographys-limits-seldom-understood/
http://nyti.ms/1kRB5ef
http://nyti.ms/OqwCkN
http://nyti.ms/1e0qqcP
http://nyti.ms/1lDFQpc
http://nyti.ms/1kReO07
http://nyti.ms/1qzNJQ9
http://nyti.ms/1oSaISU
http://t.co/shPl72Jt
http://nyti.ms/1ihOzx8
http://nyti.ms/1ihNbL0
http://nyti.ms/1ihKFnZ
http://nyti.ms/1kOSxQA
http://nyti.ms/1kOKPpM
http://well.blogs.nytimes.com/2014/03/13/emergency-rooms-

http://nyti.ms/12N0Tk5
http://nyti.ms/15B1KlO
http://nyti.ms/1cvUprv
http://nyti.ms/12Mqbij
http://nyti.ms/12MhwfB
http://nyti.ms/17ownsE
http://nyti.ms/1cvSH9s
http://nyti.ms/12LHShL
http://nyti.ms/12LFnMo
http://nyti.ms/12LwT83
http://nyti.ms/1cjFIt9
http://nyti.ms/15zu0Fi
http://nyti.ms/1cjBMbN
http://nyti.ms/18QtJiN
http://nyti.ms/13r0VfH
http://nyti.ms/13r0VfJ
http://nyti.ms/18Qgxdv
http://nyti.ms/18PZsR5
http://nyti.ms/15yJYj1
http://nyti.ms/13qjYqJ
http://nyti.ms/18PFe9X
http://nyti.ms/142RsuZ
http://nyti.ms/1csdPO2
http://nyti.ms/15uAdSQ
http://nyti.ms/1csdCdB
http://nyti.ms/1csd5ID
http://nyti.ms/13CE8ZM
http://nyti.ms/13p5FCF
http://nyti.ms/169jEen
http://nyti.ms/1bMBJG9
http://nyti.ms/134cJBb
http://nyti.ms/1cp71R1
http://nyti.ms/1codCv3
http://nyti.ms/17Uat2O
http://nyti.ms/17TRghC
http://nyti.ms/12DWgbU
http://nyti.ms/1aVRab4
http://nyti.ms/12DWgbS
http://nyti.ms/1aVRab2
http://nyti.ms/12DWf7I
http://nyti.ms/12E1K6A
http://nyti.ms/15tjCPj
http://nyti.ms/17bltGJ
http://www.

E-Cigarettes on Rise Among Teenagers, Study Finds http://nyti.ms/1szZo5Q


http://nyti.ms/1szZo5Q
Fewer Ebola Cases Go Unreported Than Thought, Study Finds  http://nyti.ms/1wbY20X


http://nyti.ms/1wbY20X
Death in Bronx Shows Vulnerability of State’s Nursing Home Residents  http://nyti.ms/13sy0eF


http://nyti.ms/13sy0eF
With Hospitals Under Stress, Tennessee’s Governor Pursues Medicaid Expansion http://nyti.ms/12ZZesm


http://nyti.ms/12ZZesm
The Mumps, a Scourge of Dormitories, Spreads Through the N.H.L. http://nyti.ms/13soZSR


http://nyti.ms/13soZSR
Paying Till It Hurts: The Odd Math of Medical Tests: One Scan, Two Prices, Both High http://nyti.ms/1GK1q4b


http://nyti.ms/1GK1q4b
Reactions: Darwin in Space, Dying Well, Eating the Cure http://nyti.ms/1Af3UFJ


http://nyti.ms/1Af3UFJ
Senate, Ending Long Fight, Confirms Obama Pick for Surgeon General http://nyti.ms/1Af82po


http://nyti.ms/1Af82po
Memo From Paris: A Plan to Limit Cars in Paris Collides With French Politics  http://nyt

...



http://nyti.ms/1cNCdIr
Well: Vitamin D Fails to Ease Winter Coughs and Colds http://nyti.ms/1cNu4nt


http://nyti.ms/1cNu4nt
Well: Palliative Care, the Treatment That Respects Pain http://nyti.ms/189Cbe7


http://nyti.ms/189Cbe7
White House Praises Gains on Health Site http://nyti.ms/18gjylC


http://nyti.ms/18gjylC
Urban Schools Aim for Environmental Revolution http://nyti.ms/1eFbkYh


http://nyti.ms/1eFbkYh
Op-Ed Contributor: Bad Eating Habits Start in the Womb http://nyti.ms/1eF6KJJ


http://nyti.ms/1eF6KJJ
Can't imagine eating anymore Thanksgiving food? Check out our Vegetarian Recipe database for some new recipes. http://nyti.ms/1c1cN89


http://nyti.ms/1c1cN89
Quiz Time: How much do you know about American families? http://nyti.ms/1c1pnnZ


http://nyti.ms/1c1pnnZ
Well: Timing of Baby’s First Solids May Affect Allergy Risk http://nyti.ms/Il0lsK


http://nyti.ms/Il0lsK
Well: Turning Leftovers Into Latkes http://nyti.ms/1b3YjXD


http://nyti.ms/1b3YjXD
RT @celiadugger: Rise in unp

Mental Health Again an Issue in Gun Debate http://nyti.ms/1gBeImG


http://nyti.ms/1gBeImG
Well: What a Messy Desk Says About You http://nyti.ms/1eSIYhk


http://nyti.ms/1eSIYhk
Your Money Adviser: Businesses Manage Medical Bill Payment Plans http://nyti.ms/18AvflM


http://nyti.ms/18AvflM
Anti-Hunger Advocates Put Pressure on Lawmakers Over Food Stamp Bill http://nyti.ms/18aurYJ


http://nyti.ms/18aurYJ
After Uproar, Penn State Suspends Penalty Fee in Wellness Plan http://nyti.ms/14k5Z5Z


http://nyti.ms/14k5Z5Z
More evidence that recess leads to school success.  http://nyti.ms/19eRfDH


http://nyti.ms/19eRfDH
$33 Million Grant for Effort to Prevent Early-Onset Alzheimer’s http://nyti.ms/1eQh2KV


http://nyti.ms/1eQh2KV
. @PamBelluck writes about $33 million worth of hope for Alzheimer's prevention. http://nyti.ms/19eXHKT


http://nyti.ms/19eXHKT
Ask an Expert: Taking Questions on Causes and Effects of Sleep Deficiency http://nyti.ms/1eQ2j2z


http://nyti.ms/1eQ2j2z
RT @paula_span: In



http://nyti.ms/15bc1Dy
The Week: A Meteor’s Shock Wave; Resisting Cholera http://nyti.ms/12hYUlu


http://nyti.ms/12hYUlu
Unlikely Partners, Freeing Chimps From the Lab http://nyti.ms/12TEYkp


http://nyti.ms/12TEYkp
What Is Nostalgia Good For? Quite a Bit, Research Shows http://nyti.ms/12hNs9B


http://nyti.ms/12hNs9B
Pollution Leads to Drop in Life Span in Northern China, Research Finds http://nyti.ms/14EYrte


http://nyti.ms/14EYrte
If you only have an hour to workout every week, are you better off doing one long workout or three shorter ones? http://well.blogs.nytimes.com/2013/07/05/ask-well-3-short-workouts-or-1-long-one/


http://well.blogs.nytimes.com/2013/07/05/ask-well-3-short-workouts-or-1-long-one/
18 and Under: Culprits in a Child’s Headaches http://nyti.ms/12h68Gk


http://nyti.ms/12h68Gk
Can one side of the body be more allergic than the other? http://nyti.ms/12g7vVU


http://nyti.ms/12g7vVU
Q&amp;A: Can One Side of the Body Be More Allergic Than the Other? http://nyti.


http://nyti.ms/12NMY8B
Navajo Confront Increase in New H.I.V. Infections http://nyti.ms/13EXtxa


http://nyti.ms/13EXtxa
Opinion: Disease and the Public Eye http://nyti.ms/15XQe7l


http://nyti.ms/15XQe7l
The Texas Tribune: With Consensus and Money, State Takes on Mental Health Care http://nyti.ms/10FFYYo


http://nyti.ms/10FFYYo
Marijuana Bill Advances in Illinois http://nyti.ms/15XQe7j


http://nyti.ms/15XQe7j
Shinseki Faces Mounting Criticism Over Backlog of Benefit Claims http://nyti.ms/10FG1U2


http://nyti.ms/10FG1U2
The Health Toll of Immigration http://nyti.ms/15XQe7f


http://nyti.ms/15XQe7f
For Gay Men, a Fear That Feels Familiar http://nyti.ms/10DUVKL


http://nyti.ms/10DUVKL
World Briefing 


Abortion Law in Arkansas Is Blocked by U.S. Judge http://nyti.ms/18RxvsC


http://nyti.ms/18RxvsC
Staying in a nonsmoking room in a hotel that allows smoking elsewhere does not prevent exposure to tobacco smoke.  http://nyti.ms/14x9hk8


http://nyti.ms/14x9hk8
Well: Focaccia Recipes f

http://nyti.ms/XOpLnY
Financial burden for dementia care is at least as high as that for heart disease or cancer, reports @PamBelluck. http://nyti.ms/14Or7SP


http://nyti.ms/14Or7SP
Straw-like tubes of modified gelatin provide protection as implanted cells grow and work in the body. http://blogs.discovermagazine.com/d-brief/2013/04/02/cell-fibers-could-be-woven-into-organ-transplants-of-the-future/#.UVxiQhmZZUE


http://blogs.discovermagazine.com/d-brief/2013/04/02/cell-fibers-could-be-woven-into-organ-transplants-of-the-future/
Mindfulness to control mind wandering? 


The New Old Age Blog: How Do You Keep Track of It All? http://nyti.ms/XNP8WW


http://nyti.ms/XNP8WW
RT @NY4P: The art of the 7am @highlinenyc jog, via @nytimeswell - http://well.blogs.nytimes.com/2013/04/02/the-roving-runner-elevating-the-jog/


http://well.blogs.nytimes.com/2013/04/02/the-roving-runner-elevating-the-jog/
AIDS Vaccine Path Suggested by Study http://nyti.ms/14ENbiA


http://nyti.ms/14ENbiA
Well: How Me

In [6]:
frame.head()

,tweetid,date,title,url
0,548662191340421120,Sat Dec 27 02:10:34 +0000 2014,Risks in Using Social Media to Spot Signs of M...,http://nyti.ms/1rqi9I1
1,548579831169163265,Fri Dec 26 20:43:18 +0000 2014,RT @paula_span: The most effective nationwide ...,http://newoldage.blogs.nytimes.com/2014/12/26/...
2,548579045269852161,Fri Dec 26 20:40:11 +0000 2014,The New Old Age Blog: Diabetes Prevention That...,http://nyti.ms/1xm7fTi
3,548444679529041920,Fri Dec 26 11:46:15 +0000 2014,Well: Comfort Casseroles for Winter Dinners,http://nyti.ms/1xTNoO0
4,548311901227474944,Fri Dec 26 02:58:39 +0000 2014,High-Level Knowledge Before Veterans Affairs S...,http://nyti.ms/13yCpvS


In [7]:
frame.loc[:,'title'][0]

'Risks in Using Social Media to Spot Signs of Mental Distress '

In [8]:
# Notes: Well, Science, Technology
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//div[@class="entry-content"]/p[@class="story-body-text"]/text()',
                   '',
                   '//header/div[@class="story-meta-footer"]/address/text()']
list_scrapt_mainlink= ['//section/div/div/p/text()',
                          ''
                            '//*[@id="story"]/header/div[4]/div/div/div/p/a/span']

In [9]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [10]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class="entry-content"]/p[@class="story-body-text"]/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//header/div[@class="story-meta-footer"]/address/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://nyti.ms/1rqi9I1
0
0
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  0
tags  :  0
author  :  0




scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://newoldage.blogs.nytimes.com/2014/12/26/diabetes-prevention-that-works/
6059
21
contents  :  6059
tags  :  0
author  :  21


Header:   200
http://nyti.ms/1xm7fTi
6059
21
contents  :  6059
tags  :  0
author  :  21


Header:   200
http://nyti.ms/1xTNoO0
1961
30
contents  :  1961
tags  :  0
author  :  30


Header:   200
http://nyti.ms/13yCpvS
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/13yAtUf
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1vtPNcm
3793
38
contents  :  3793
tags  :  0
author  :  38


Header:   200
http://nyti.ms/1wLtAen
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/1ziCflj
9562
27
contents  :  9562
tags  :  0
author  :  27


Header:   200
http://nyti.ms/1xNzyg4
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1vpIjqE
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/16RCgpE
0
0
contents  :  0
tags  :  0
author  :  0


Head

Header:   200
http://nyti.ms/136pRLR
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1yOAMI9
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1zhBhs0
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1zHKkDH
1595
27
contents  :  1595
tags  :  0
author  :  27


Header:   200
http://nyti.ms/1wn60mf
5578
21
contents  :  5578
tags  :  0
author  :  21


Header:   200
http://nyti.ms/1DmTjy0
2803
30
contents  :  2803
tags  :  0
author  :  30


Header:   200
http://nyti.ms/1GiLCqV
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1shJpcB
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1GiLAiz
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://bit.ly/1Bj4hzW
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1uqctsD
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/16c5FdP
0
0
contents  :  0
tags  :  0
author  

Header:   200
http://nyti.ms/1ye3u4Y
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1ye10na
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1vgQ5lD
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1FL3NU1
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1rSOkjZ
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1FL3Lvk
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1tBtT60
1354
27
contents  :  1354
tags  :  0
author  :  27


Header:   200
http://nyti.ms/1wkSger
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1wkRe1O
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1HVoZui
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/11OOaO7
4690
27
contents  :  4690
tags  :  0
author  :  27


Header:   200
http://nyti.ms/1FJC3Pv
4690
27
contents  :  4690
tags  :  0
author  

...

Header:   200
http://nyti.ms/12bjUZI
5442
25
contents  :  5442
tags  :  0
author  :  25


Header:   200
http://nyti.ms/12eCw9V
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/12ekTHb
5442
25
contents  :  5442
tags  :  0
author  :  25


Header:   200
http://nyti.ms/17GXMIL
5401
26
contents  :  5401
tags  :  0
author  :  26


Header:   200
http://nyti.ms/15uyArf
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/18Sh7Wa
5466
32
contents  :  5466
tags  :  0
author  :  32


Header:   200
http://nyti.ms/Zqdh2u
6298
32
contents  :  6298
tags  :  0
author  :  32


Header:   200
http://nyti.ms/15vhCJr
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/15v0inT
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/129qHmt
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/15v0inQ
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/129qFLz
0
0
contents  :  0
tag

Header:   200
http://nyti.ms/14LdIf0
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/ZhvzTh
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/ZRU7FC
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/12Lqru8
1542
26
contents  :  1542
tags  :  0
author  :  26


Header:   200
http://nyti.ms/12LfsAT
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/12KY3bq
3892
27
contents  :  3892
tags  :  0
author  :  27


Header:   200
http://nyti.ms/Y9vm7r
4977
27
contents  :  4977
tags  :  0
author  :  27


Header:   200
http://nyti.ms/154m9CB
1369
27
contents  :  1369
tags  :  0
author  :  27


Header:   200
http://nyti.ms/Y9ry66
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/Y9goyc
5385
27
contents  :  5385
tags  :  0
author  :  27


Header:   200
http://nyti.ms/12KknBT
4968
36
contents  :  4968
tags  :  0
author  :  36


Header:   200
http://nyti.ms/12KbYOU
0
0
contents  :  0

Header:   200
http://nyti.ms/XR7OnJ
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/11trBtJ
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/11sixnN
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/XXz8CQ
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/11sivwd
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/XXzaKS
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/11rZXfB
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/XXoSKI
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/11rUi9n
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/17yEvY1
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/17ywCSu
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/11iH1B0
4471
26
contents  :  4471
tags  :  0
author  :  26


Header:   20

Header:   200
http://nyti.ms/17udLeb
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/1109OKa
1397
27
contents  :  1397
tags  :  0
author  :  27


Header:   200
http://nyti.ms/Zc8vbi
6675
24
contents  :  6675
tags  :  0
author  :  24


Header:   200
http://nyti.ms/ZpgXjH
1461
27
contents  :  1461
tags  :  0
author  :  27


Header:   200
http://nyti.ms/YNTWv6
1461
27
contents  :  1461
tags  :  0
author  :  27


Header:   200
http://nyti.ms/10Oj0A8
5404
30
contents  :  5404
tags  :  0
author  :  30


Header:   200
http://t.co
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/ZO98nt
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/154Zk0M
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/YME5wX
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/10MQkch
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/14ciFNP
1360
27
contents  :  1360
tags  :  0


Header:   200
http://nyti.ms/10t5rGq
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/10wfjxA
5688
25
contents  :  5688
tags  :  0
author  :  25


Header:   200
http://nyti.ms/11htuuj
5121
21
contents  :  5121
tags  :  0
author  :  21


Header:   200
http://nyti.ms/100BTRt
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/14B7AVH
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/14AYNmz
3397
27
contents  :  3397
tags  :  0
author  :  27


Header:   200
http://nyti.ms/14AWVKt
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/YXtcVa
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/14AEpCc
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/YX5IzE
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/14AGluo
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://nyti.ms/14AMtTm
0
0
contents  :  0
tags  :  0
author  :

In [11]:
# -------------------------------------------------
frame.head()

,author,contents,date,tags,title,tweetid,url
0,,,Sat Dec 27 02:10:34 +0000 2014,,Risks in Using Social Media to Spot Signs of M...,5.486622e+17,http://nyti.ms/1rqi9I1
1,\n\t\t\t\tBy Paula Span\t\t\t,What happens in Room 6 at the Y.M.C.A. in Wayn...,Fri Dec 26 20:43:18 +0000 2014,,RT @paula_span: The most effective nationwide ...,5.485798e+17,http://newoldage.blogs.nytimes.com/2014/12/26/...
2,\n\t\t\t\tBy Paula Span\t\t\t,What happens in Room 6 at the Y.M.C.A. in Wayn...,Fri Dec 26 20:40:11 +0000 2014,,The New Old Age Blog: Diabetes Prevention That...,5.485790e+17,http://nyti.ms/1xm7fTi
3,\n\t\t\t\tBy Martha Rose Shulman\t\t\t,"When I was in high school, way back in the day...",Fri Dec 26 11:46:15 +0000 2014,,Well: Comfort Casseroles for Winter Dinners,5.484447e+17,http://nyti.ms/1xTNoO0
4,,,Fri Dec 26 02:58:39 +0000 2014,,High-Level Knowledge Before Veterans Affairs S...,5.483119e+17,http://nyti.ms/13yCpvS


In [12]:
# -------------------------------------------------
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6045 entries, 0 to 6044
Data columns (total 7 columns):
author      5543 non-null object
contents    5543 non-null object
date        6045 non-null object
tags        5543 non-null object
title       6045 non-null object
tweetid     6045 non-null float64
url         6045 non-null object
dtypes: float64(1), object(6)
memory usage: 537.8+ KB


In [13]:
frame.to_csv(dest_url, sep='|', encoding='utf-8',index=False)

In [3]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_csv(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
#     success_date = './date/success_date.txt'
    reg_special = r"([^a-zA-Z\s]+)"
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Clean content a bit
            contents = re.sub(reg_special, '', contents)
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
#             f1 = open(success_date, "a")
#             f1.write(frame['date'].iloc[key])
#             f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  3
[Out] key:  0
1  :  6059
2  :  6059
3  :  1976
4  :  3
[Out] key:  4
5  :  3
[Out] key:  5
6  :  3793
7  :  3
[Out] key:  7
8  :  9562
9  :  3
[Out] key:  9
10  :  3
[Out] key:  10
11  :  3
[Out] key:  11
12  :  3
[Out] key:  12
13  :  3
[Out] key:  13
14  :  3
[Out] key:  14
15  :  9562
16  :  3
[Out] key:  16
17  :  5298
18  :  3
[Out] key:  18
19  :  3
[Out] key:  19
20  :  9656
21  :  3
[Out] key:  21
22  :  3
[Out] key:  22
23  :  3
[Out] key:  23
24  :  1390
25  :  3
[Out] key:  25
26  :  1349
27  :  5187
28  :  3
[Out] key:  28
29  :  3
[Out] key:  29
30  :  3
[Out] key:  30
31  :  5187
32  :  3
[Out] key:  32
33  :  3
[Out] key:  33
34  :  3
[Out] key:  34
35  :  3
[Out] key:  35
36  :  3
[Out] key:  36
37  :  3
[Out] key:  37
38  :  3
[Out] key:  38
39  :  3
[Out] key:  39
40  :  3
[Out] key:  40
41  :  3
[Out] key:  41
42  :  3
[Out] key:  42
43  :  4971
44  :  1390
45  :  3
[Out] key:  45
46  :  3
[Out] key:  46
47  :  6326
48  :  3
[Out] key:  48
49  :  3
[Out] key:

479  :  3
[Out] key:  479
480  :  3
[Out] key:  480
481  :  3
[Out] key:  481
482  :  3
[Out] key:  482
483  :  1456
484  :  1463
485  :  4836
486  :  3
[Out] key:  486
487  :  3
[Out] key:  487
488  :  3
[Out] key:  488
489  :  3
[Out] key:  489
490  :  3
[Out] key:  490
491  :  1348
492  :  4985
493  :  3
[Out] key:  493
494  :  1303
495  :  6222
496  :  4985
497  :  3
[Out] key:  497
498  :  3
[Out] key:  498
499  :  4985
500  :  3
[Out] key:  500
501  :  3
[Out] key:  501
502  :  3
[Out] key:  502
503  :  3
[Out] key:  503
504  :  3
[Out] key:  504
505  :  3
[Out] key:  505
506  :  3
[Out] key:  506
507  :  3
[Out] key:  507
508  :  3
[Out] key:  508
509  :  3
[Out] key:  509
510  :  3
[Out] key:  510
511  :  3
[Out] key:  511
512  :  4775
513  :  3
[Out] key:  513
514  :  2040
515  :  3
[Out] key:  515
516  :  3
[Out] key:  516
517  :  3
[Out] key:  517
518  :  3
[Out] key:  518
519  :  6269
520  :  3
[Out] key:  520
521  :  3
[Out] key:  521
522  :  3
[Out] key:  522
523  :  3
[O

921  :  3
[Out] key:  921
922  :  3
[Out] key:  922
923  :  3
[Out] key:  923
924  :  3
[Out] key:  924
925  :  3
[Out] key:  925
926  :  3
[Out] key:  926
927  :  1219
928  :  1435
929  :  3
[Out] key:  929
930  :  3379
931  :  4975
932  :  3
[Out] key:  932
933  :  3
[Out] key:  933
934  :  6511
935  :  3
[Out] key:  935
936  :  5594
937  :  3
[Out] key:  937
938  :  3
[Out] key:  938
939  :  2397
940  :  3
[Out] key:  940
941  :  3378
942  :  3
[Out] key:  942
943  :  3
[Out] key:  943
944  :  1254
945  :  3
[Out] key:  945
946  :  3
[Out] key:  946
947  :  3
[Out] key:  947
948  :  3
[Out] key:  948
949  :  3
[Out] key:  949
950  :  3
[Out] key:  950
951  :  5766
952  :  3
[Out] key:  952
953  :  3
[Out] key:  953
954  :  3
[Out] key:  954
955  :  3
[Out] key:  955
956  :  3
[Out] key:  956
957  :  3
[Out] key:  957
958  :  3
[Out] key:  958
959  :  3
[Out] key:  959
960  :  5840
961  :  1254
962  :  3
[Out] key:  962
963  :  3
[Out] key:  963
964  :  3
[Out] key:  964
965  :  5614

1377  :  1320
1378  :  3
[Out] key:  1378
1379  :  3
[Out] key:  1379
1380  :  1320
1381  :  3
[Out] key:  1381
1382  :  1320
1383  :  5766
1384  :  1320
1385  :  3
[Out] key:  1385
1386  :  3
[Out] key:  1386
1387  :  3
[Out] key:  1387
1388  :  3345
1389  :  3
[Out] key:  1389
1390  :  3
[Out] key:  1390
1391  :  3
[Out] key:  1391
1392  :  3
[Out] key:  1392
1393  :  3
[Out] key:  1393
1394  :  5915
1395  :  5915
1396  :  3
[Out] key:  1396
1397  :  3
[Out] key:  1397
1398  :  5766
1399  :  3
[Out] key:  1399
1400  :  3
[Out] key:  1400
1401  :  3
[Out] key:  1401
1402  :  4824
1403  :  5119
1404  :  4824
1405  :  3
[Out] key:  1405
1406  :  3
[Out] key:  1406
1407  :  3
[Out] key:  1407
1408  :  3
[Out] key:  1408
1409  :  3
[Out] key:  1409
1410  :  3
[Out] key:  1410
1411  :  3
[Out] key:  1411
1412  :  5915
1413  :  3
[Out] key:  1413
1414  :  3
[Out] key:  1414
1415  :  3
[Out] key:  1415
1416  :  5915
1417  :  5915
1418  :  5915
1419  :  3
[Out] key:  1419
1420  :  3
[Out] key

1802  :  4310
1803  :  3
[Out] key:  1803
1804  :  3
[Out] key:  1804
1805  :  3
[Out] key:  1805
1806  :  3
[Out] key:  1806
1807  :  3
[Out] key:  1807
1808  :  3
[Out] key:  1808
1809  :  3
[Out] key:  1809
1810  :  1957
1811  :  3
[Out] key:  1811
1812  :  3
[Out] key:  1812
1813  :  3
[Out] key:  1813
1814  :  3
[Out] key:  1814
1815  :  3
[Out] key:  1815
1816  :  3
[Out] key:  1816
1817  :  3
[Out] key:  1817
1818  :  3
[Out] key:  1818
1819  :  3
[Out] key:  1819
1820  :  3
[Out] key:  1820
1821  :  3
[Out] key:  1821
1822  :  3
[Out] key:  1822
1823  :  3
[Out] key:  1823
1824  :  4310
1825  :  2292
1826  :  3
[Out] key:  1826
1827  :  1181
1828  :  1380
1829  :  3
[Out] key:  1829
1830  :  1181
1831  :  8093
1832  :  3
[Out] key:  1832
1833  :  3
[Out] key:  1833
1834  :  3
[Out] key:  1834
1835  :  3
[Out] key:  1835
1836  :  3
[Out] key:  1836
1837  :  1368
1838  :  1368
1839  :  1344
1840  :  3
[Out] key:  1840
1841  :  3
[Out] key:  1841
1842  :  3
[Out] key:  1842
1843  

...

4112  :  3
[Out] key:  4112
4113  :  5742
4114  :  1266
4115  :  3
[Out] key:  4115
4116  :  3
[Out] key:  4116
4117  :  4816
4118  :  1511
4119  :  63
4120  :  3
[Out] key:  4120
4121  :  3
[Out] key:  4121
4122  :  4634
4123  :  3
[Out] key:  4123
4124  :  3
[Out] key:  4124
4125  :  3463
4126  :  1311
4127  :  3
[Out] key:  4127
4128  :  4792
4129  :  1403
4130  :  3
[Out] key:  4130
4131  :  3
[Out] key:  4131
4132  :  1403
4133  :  4080
4134  :  3463
4135  :  3
[Out] key:  4135
4136  :  3
[Out] key:  4136
4137  :  3
[Out] key:  4137
4138  :  3
[Out] key:  4138
4139  :  3
[Out] key:  4139
4140  :  7007
4141  :  5289
4142  :  3
[Out] key:  4142
4143  :  3
[Out] key:  4143
4144  :  5052
4145  :  3
[Out] key:  4145
4146  :  5289
4147  :  3
[Out] key:  4147
4148  :  3
[Out] key:  4148
4149  :  3
[Out] key:  4149
4150  :  3
[Out] key:  4150
4151  :  4946
4152  :  3169
4153  :  5052
4154  :  3
[Out] key:  4154
4155  :  3
[Out] key:  4155
4156  :  3
[Out] key:  4156
4157  :  3
[Out] key: 

4512  :  3891
4513  :  514
4514  :  4666
4515  :  3220
4516  :  4987
4517  :  3220
4518  :  3
[Out] key:  4518
4519  :  3
[Out] key:  4519
4520  :  4987
4521  :  8601
4522  :  4666
4523  :  3
[Out] key:  4523
4524  :  4352
4525  :  1366
4526  :  8601
4527  :  3
[Out] key:  4527
4528  :  3
[Out] key:  4528
4529  :  3
[Out] key:  4529
4530  :  3
[Out] key:  4530
4531  :  3891
4532  :  3891
4533  :  5223
4534  :  3
[Out] key:  4534
4535  :  5691
4536  :  1498
4537  :  1317
4538  :  3
[Out] key:  4538
4539  :  5223
4540  :  3
[Out] key:  4540
4541  :  5528
4542  :  1277
4543  :  4790
4544  :  3
[Out] key:  4544
4545  :  5691
4546  :  5528
4547  :  3
[Out] key:  4547
4548  :  3
[Out] key:  4548
4549  :  3
[Out] key:  4549
4550  :  1498
4551  :  3
[Out] key:  4551
4552  :  1498
4553  :  5528
4554  :  5691
4555  :  3
[Out] key:  4555
4556  :  3
[Out] key:  4556
4557  :  3
[Out] key:  4557
4558  :  1184
4559  :  3
[Out] key:  4559
4560  :  1957
4561  :  1277
4562  :  5134
4563  :  1957
4564  :

4955  :  3
[Out] key:  4955
4956  :  3
[Out] key:  4956
4957  :  3
[Out] key:  4957
4958  :  3
[Out] key:  4958
4959  :  3
[Out] key:  4959
4960  :  3
[Out] key:  4960
4961  :  66
4962  :  3
[Out] key:  4962
4963  :  3
[Out] key:  4963
4964  :  5263
4965  :  3
[Out] key:  4965
4966  :  5114
4967  :  3
[Out] key:  4967
4968  :  3
[Out] key:  4968
4969  :  1307
4970  :  5263
4971  :  3
[Out] key:  4971
4972  :  3
[Out] key:  4972
4973  :  3
[Out] key:  4973
4974  :  5237
4975  :  3
[Out] key:  4975
4976  :  3
[Out] key:  4976
4977  :  3
[Out] key:  4977
4978  :  3
[Out] key:  4978
4979  :  5503
4980  :  3
[Out] key:  4980
4981  :  1381
4982  :  6379
4983  :  3
[Out] key:  4983
4984  :  3
[Out] key:  4984
4985  :  3
[Out] key:  4985
4986  :  3
[Out] key:  4986
4987  :  3
[Out] key:  4987
4988  :  3
[Out] key:  4988
4989  :  3
[Out] key:  4989
4990  :  5503
4991  :  3
[Out] key:  4991
4992  :  3
[Out] key:  4992
4993  :  4586
4994  :  3
[Out] key:  4994
4995  :  6232
4996  :  3
[Out] key: 

5419  :  5703
5420  :  5384
5421  :  1551
5422  :  3
[Out] key:  5422
5423  :  4594
5424  :  3
[Out] key:  5424
5425  :  3
[Out] key:  5425
5426  :  3
[Out] key:  5426
5427  :  3
[Out] key:  5427
5428  :  3
[Out] key:  5428
5429  :  3
[Out] key:  5429
5430  :  3
[Out] key:  5430
5431  :  3
[Out] key:  5431
5432  :  5384
5433  :  3
[Out] key:  5433
5434  :  3
[Out] key:  5434
5435  :  1551
5436  :  3
[Out] key:  5436
5437  :  3
[Out] key:  5437
5438  :  4594
5439  :  3
[Out] key:  5439
5440  :  6239
5441  :  3
[Out] key:  5441
5442  :  3
[Out] key:  5442
5443  :  3
[Out] key:  5443
5444  :  3
[Out] key:  5444
5445  :  3
[Out] key:  5445
5446  :  3
[Out] key:  5446
5447  :  3
[Out] key:  5447
5448  :  3
[Out] key:  5448
5449  :  3
[Out] key:  5449
5450  :  646
5451  :  1128
5452  :  5873
5453  :  59
5454  :  3
[Out] key:  5454
5455  :  3
[Out] key:  5455
5456  :  3
[Out] key:  5456
5457  :  3
[Out] key:  5457
5458  :  5873
5459  :  4942
5460  :  4942
5461  :  4705
5462  :  1351
5463  :  

5915  :  3
[Out] key:  5915
5916  :  3
[Out] key:  5916
5917  :  3
[Out] key:  5917
5918  :  3
[Out] key:  5918
5919  :  3
[Out] key:  5919
5920  :  3
[Out] key:  5920
5921  :  595
5922  :  3
[Out] key:  5922
5923  :  6
[Out] key:  5923
5924  :  3
[Out] key:  5924
5925  :  4482
5926  :  3
[Out] key:  5926
5927  :  3
[Out] key:  5927
5928  :  3
[Out] key:  5928
5929  :  3
[Out] key:  5929
5930  :  1397
5931  :  6675
5932  :  1461
5933  :  1461
5934  :  5405
5935  :  3
[Out] key:  5935
5936  :  3
[Out] key:  5936
5937  :  3
[Out] key:  5937
5938  :  3
[Out] key:  5938
5939  :  3
[Out] key:  5939
5940  :  1360
5941  :  3
[Out] key:  5941
5942  :  3
[Out] key:  5942
5943  :  3
[Out] key:  5943
5944  :  6444
5945  :  3
[Out] key:  5945
5946  :  3
[Out] key:  5946
5947  :  6444
5948  :  5660
5949  :  3
[Out] key:  5949
5950  :  3
[Out] key:  5950
5951  :  3
[Out] key:  5951
5952  :  3
[Out] key:  5952
5953  :  3268
5954  :  1349
5955  :  3
[Out] key:  5955
5956  :  3
[Out] key:  5956
5957  :

In [14]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
    success_date = './date/success_date.txt'
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
            f1 = open(success_date, "a")
            f1.write(frame['date'].iloc[key])
            f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  3
[Out] key:  0
1  :  6059
2  :  6059
3  :  1976
4  :  3
[Out] key:  4
5  :  3
[Out] key:  5
6  :  3793
7  :  3
[Out] key:  7
8  :  9562
9  :  3
[Out] key:  9
10  :  3
[Out] key:  10
11  :  3
[Out] key:  11
12  :  3
[Out] key:  12
13  :  3
[Out] key:  13
14  :  3
[Out] key:  14
15  :  9562
16  :  3
[Out] key:  16
17  :  5298
18  :  3
[Out] key:  18
19  :  3
[Out] key:  19
20  :  9656
21  :  3
[Out] key:  21
22  :  3
[Out] key:  22
23  :  3
[Out] key:  23
24  :  1390
25  :  3
[Out] key:  25
26  :  1349
27  :  5187
28  :  3
[Out] key:  28
29  :  3
[Out] key:  29
30  :  3
[Out] key:  30
31  :  5187
32  :  3
[Out] key:  32
33  :  3
[Out] key:  33
34  :  3
[Out] key:  34
35  :  3
[Out] key:  35
36  :  3
[Out] key:  36
37  :  3
[Out] key:  37
38  :  3
[Out] key:  38
39  :  3
[Out] key:  39
40  :  3
[Out] key:  40
41  :  3
[Out] key:  41
42  :  3
[Out] key:  42
43  :  4971
44  :  1390
45  :  3
[Out] key:  45
46  :  3
[Out] key:  46
47  :  6326
48  :  3
[Out] key:  48
49  :  3
[Out] key:

502  :  3
[Out] key:  502
503  :  3
[Out] key:  503
504  :  3
[Out] key:  504
505  :  3
[Out] key:  505
506  :  3
[Out] key:  506
507  :  3
[Out] key:  507
508  :  3
[Out] key:  508
509  :  3
[Out] key:  509
510  :  3
[Out] key:  510
511  :  3
[Out] key:  511
512  :  4775
513  :  3
[Out] key:  513
514  :  2040
515  :  3
[Out] key:  515
516  :  3
[Out] key:  516
517  :  3
[Out] key:  517
518  :  3
[Out] key:  518
519  :  6269
520  :  3
[Out] key:  520
521  :  3
[Out] key:  521
522  :  3
[Out] key:  522
523  :  3
[Out] key:  523
524  :  3
[Out] key:  524
525  :  199
526  :  3
[Out] key:  526
527  :  3
[Out] key:  527
528  :  3
[Out] key:  528
529  :  3
[Out] key:  529
530  :  3
[Out] key:  530
531  :  3
[Out] key:  531
532  :  3
[Out] key:  532
533  :  3
[Out] key:  533
534  :  3
[Out] key:  534
535  :  8475
536  :  3
[Out] key:  536
537  :  1392
538  :  3
[Out] key:  538
539  :  3
[Out] key:  539
540  :  3
[Out] key:  540
541  :  5322
542  :  3
[Out] key:  542
543  :  3
[Out] key:  543


[Out] key:  1146
1147  :  3
[Out] key:  1147
1148  :  3
[Out] key:  1148
1149  :  3
[Out] key:  1149
1150  :  3
[Out] key:  1150
1151  :  3
[Out] key:  1151
1152  :  3
[Out] key:  1152
1153  :  3
[Out] key:  1153
1154  :  3
[Out] key:  1154
1155  :  3
[Out] key:  1155
1156  :  3
[Out] key:  1156
1157  :  3
[Out] key:  1157
1158  :  3
[Out] key:  1158
1159  :  3
[Out] key:  1159
1160  :  3
[Out] key:  1160
1161  :  3
[Out] key:  1161
1162  :  3
[Out] key:  1162
1163  :  3
[Out] key:  1163
1164  :  3
[Out] key:  1164
1165  :  3
[Out] key:  1165
1166  :  1765
1167  :  389
1168  :  6311
1169  :  1304
1170  :  3
[Out] key:  1170
1171  :  3
[Out] key:  1171
1172  :  3
[Out] key:  1172
1173  :  3
[Out] key:  1173
1174  :  3
[Out] key:  1174
1175  :  3
[Out] key:  1175
1176  :  3
[Out] key:  1176
1177  :  3
[Out] key:  1177
1178  :  3
[Out] key:  1178
1179  :  3
[Out] key:  1179
1180  :  3
[Out] key:  1180
1181  :  3
[Out] key:  1181
1182  :  3
[Out] key:  1182
1183  :  3
[Out] key:  1183
1184

1812  :  3
[Out] key:  1812
1813  :  3
[Out] key:  1813
1814  :  3
[Out] key:  1814
1815  :  3
[Out] key:  1815
1816  :  3
[Out] key:  1816
1817  :  3
[Out] key:  1817
1818  :  3
[Out] key:  1818
1819  :  3
[Out] key:  1819
1820  :  3
[Out] key:  1820
1821  :  3
[Out] key:  1821
1822  :  3
[Out] key:  1822
1823  :  3
[Out] key:  1823
1824  :  4310
1825  :  2292
1826  :  3
[Out] key:  1826
1827  :  1181
1828  :  1380
1829  :  3
[Out] key:  1829
1830  :  1181
1831  :  8093
1832  :  3
[Out] key:  1832
1833  :  3
[Out] key:  1833
1834  :  3
[Out] key:  1834
1835  :  3
[Out] key:  1835
1836  :  3
[Out] key:  1836
1837  :  1368
1838  :  1368
1839  :  1344
1840  :  3
[Out] key:  1840
1841  :  3
[Out] key:  1841
1842  :  3
[Out] key:  1842
1843  :  3
[Out] key:  1843
1844  :  4944
1845  :  3
[Out] key:  1845
1846  :  3
[Out] key:  1846
1847  :  3
[Out] key:  1847
1848  :  3
[Out] key:  1848
1849  :  5788
1850  :  3
[Out] key:  1850
1851  :  5099
1852  :  5099
1853  :  3
[Out] key:  1853
1854  

2483  :  5094
2484  :  3
[Out] key:  2484
2485  :  6335
2486  :  3
[Out] key:  2486
2487  :  3
[Out] key:  2487
2488  :  3
[Out] key:  2488
2489  :  3
[Out] key:  2489
2490  :  1283
2491  :  3
[Out] key:  2491
2492  :  3
[Out] key:  2492
2493  :  3
[Out] key:  2493
2494  :  5414
2495  :  3
[Out] key:  2495
2496  :  3
[Out] key:  2496
2497  :  3
[Out] key:  2497
2498  :  3
[Out] key:  2498
2499  :  3
[Out] key:  2499
2500  :  3
[Out] key:  2500
2501  :  277
2502  :  1918
2503  :  3
[Out] key:  2503
2504  :  3
[Out] key:  2504
2505  :  3
[Out] key:  2505
2506  :  3
[Out] key:  2506
2507  :  1918
2508  :  3
[Out] key:  2508
2509  :  3
[Out] key:  2509
2510  :  3
[Out] key:  2510
2511  :  3
[Out] key:  2511
2512  :  3
[Out] key:  2512
2513  :  3
[Out] key:  2513
2514  :  4195
2515  :  3
[Out] key:  2515
2516  :  3
[Out] key:  2516
2517  :  3
[Out] key:  2517
2518  :  4195
2519  :  3
[Out] key:  2519
2520  :  3554
2521  :  4195
2522  :  2425
2523  :  3
[Out] key:  2523
2524  :  3
[Out] key:

3183  :  3
[Out] key:  3183
3184  :  11306
3185  :  11306
3186  :  2
[Out] key:  3186
3187  :  3
[Out] key:  3187
3188  :  11306
3189  :  5388
3190  :  1288
3191  :  5832
3192  :  7413
3193  :  6289
3194  :  1330
3195  :  6289
3196  :  7646
3197  :  5586
3198  :  3
[Out] key:  3198
3199  :  3
[Out] key:  3199
3200  :  6241
3201  :  5345
3202  :  4887
3203  :  3
[Out] key:  3203
3204  :  3
[Out] key:  3204
3205  :  6525
3206  :  3
[Out] key:  3206
3207  :  3
[Out] key:  3207
3208  :  5345
3209  :  3
[Out] key:  3209
3210  :  6241
3211  :  2517
3212  :  599
3213  :  3
[Out] key:  3213
3214  :  3
[Out] key:  3214
3215  :  7646
3216  :  6954
3217  :  6558
3218  :  6054
3219  :  4887
3220  :  6763
3221  :  6103
3222  :  3
[Out] key:  3222
3223  :  7646
3224  :  6054
3225  :  4887
3226  :  599
3227  :  6558
3228  :  7413
3229  :  5832
3230  :  2517
3231  :  6954
3232  :  6241
3233  :  7008
3234  :  3
[Out] key:  3234
3235  :  3
[Out] key:  3235
3236  :  7008
3237  :  5601
3238  :  1191
3239 

3842  :  59
3843  :  3163
3844  :  3
[Out] key:  3844
3845  :  3
[Out] key:  3845
3846  :  6927
3847  :  4427
3848  :  3
[Out] key:  3848
3849  :  5197
3850  :  3
[Out] key:  3850
3851  :  5197
3852  :  3
[Out] key:  3852
3853  :  3
[Out] key:  3853
3854  :  3
[Out] key:  3854
3855  :  3
[Out] key:  3855
3856  :  3
[Out] key:  3856
3857  :  3
[Out] key:  3857
3858  :  11518
3859  :  1812
3860  :  2764
3861  :  3
[Out] key:  3861
3862  :  3
[Out] key:  3862
3863  :  3
[Out] key:  3863
3864  :  3
[Out] key:  3864
3865  :  3
[Out] key:  3865
3866  :  7927
3867  :  4329
3868  :  3
[Out] key:  3868
3869  :  3
[Out] key:  3869
3870  :  3
[Out] key:  3870
3871  :  3
[Out] key:  3871
3872  :  4329
3873  :  5010
3874  :  3
[Out] key:  3874
3875  :  3
[Out] key:  3875
3876  :  3
[Out] key:  3876
3877  :  4853
3878  :  4853
3879  :  3
[Out] key:  3879
3880  :  2056
3881  :  5482
3882  :  5055
3883  :  5482
3884  :  2056
3885  :  3
[Out] key:  3885
3886  :  3
[Out] key:  3886
3887  :  3
[Out] key:

4458  :  5019
4459  :  1834
4460  :  5019
4461  :  1834
4462  :  3
[Out] key:  4462
4463  :  1834
4464  :  3
[Out] key:  4464
4465  :  3
[Out] key:  4465
4466  :  3
[Out] key:  4466
4467  :  3
[Out] key:  4467
4468  :  3
[Out] key:  4468
4469  :  3
[Out] key:  4469
4470  :  59
4471  :  59
4472  :  4650
4473  :  3
[Out] key:  4473
4474  :  3
[Out] key:  4474
4475  :  3
[Out] key:  4475
4476  :  4191
4477  :  5639
4478  :  1209
4479  :  3
[Out] key:  4479
4480  :  3
[Out] key:  4480
4481  :  3
[Out] key:  4481
4482  :  5639
4483  :  3
[Out] key:  4483
4484  :  3
[Out] key:  4484
4485  :  3
[Out] key:  4485
4486  :  3
[Out] key:  4486
4487  :  3
[Out] key:  4487
4488  :  4156
4489  :  3
[Out] key:  4489
4490  :  3
[Out] key:  4490
4491  :  3
[Out] key:  4491
4492  :  3
[Out] key:  4492
4493  :  3
[Out] key:  4493
4494  :  4156
4495  :  4156
4496  :  3
[Out] key:  4496
4497  :  3
[Out] key:  4497
4498  :  3
[Out] key:  4498
4499  :  3
[Out] key:  4499
4500  :  3
[Out] key:  4500
4501  :  3

4914  :  4913
4915  :  3
[Out] key:  4915
4916  :  3
[Out] key:  4916
4917  :  1668
4918  :  3
[Out] key:  4918
4919  :  3
[Out] key:  4919
4920  :  3
[Out] key:  4920
4921  :  3
[Out] key:  4921
4922  :  3
[Out] key:  4922
4923  :  3
[Out] key:  4923
4924  :  3
[Out] key:  4924
4925  :  3
[Out] key:  4925
4926  :  3
[Out] key:  4926
4927  :  3
[Out] key:  4927
4928  :  4913
4929  :  6685
4930  :  3
[Out] key:  4930
4931  :  3
[Out] key:  4931
4932  :  3
[Out] key:  4932
4933  :  686
4934  :  1340
4935  :  1668
4936  :  6287
4937  :  3
[Out] key:  4937
4938  :  6287
4939  :  3
[Out] key:  4939
4940  :  3
[Out] key:  4940
4941  :  3
[Out] key:  4941
4942  :  3
[Out] key:  4942
4943  :  3
[Out] key:  4943
4944  :  3
[Out] key:  4944
4945  :  3
[Out] key:  4945
4946  :  5530
4947  :  686
4948  :  66
4949  :  4466
4950  :  6350
4951  :  3
[Out] key:  4951
4952  :  6350
4953  :  3
[Out] key:  4953
4954  :  3
[Out] key:  4954
4955  :  3
[Out] key:  4955
4956  :  3
[Out] key:  4956
4957  :  3

5523  :  1433
5524  :  3
[Out] key:  5524
5525  :  59
5526  :  5225
5527  :  3
[Out] key:  5527
5528  :  3
[Out] key:  5528
5529  :  3
[Out] key:  5529
5530  :  3
[Out] key:  5530
5531  :  4320
5532  :  5225
5533  :  4290
5534  :  3
[Out] key:  5534
5535  :  3
[Out] key:  5535
5536  :  3
[Out] key:  5536
5537  :  3
[Out] key:  5537
5538  :  3
[Out] key:  5538
5539  :  1306
5540  :  5132
5541  :  3
[Out] key:  5541
5542  :  5440
5543  :  5440
5544  :  3
[Out] key:  5544
5545  :  5492
5546  :  5132
5547  :  3
[Out] key:  5547
5548  :  3
[Out] key:  5548
5549  :  3
[Out] key:  5549
5550  :  3
[Out] key:  5550
5551  :  3
[Out] key:  5551
5552  :  3
[Out] key:  5552
5553  :  4944
5554  :  3
[Out] key:  5554
5555  :  3
[Out] key:  5555
5556  :  3
[Out] key:  5556
5557  :  3
[Out] key:  5557
5558  :  3
[Out] key:  5558
5559  :  3
[Out] key:  5559
5560  :  5579
5561  :  8279
5562  :  6493
5563  :  1648
5564  :  6481
5565  :  4705
5566  :  3
[Out] key:  5566
5567  :  3
[Out] key:  5567
5568  : 

In [15]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
### Writing content into text file
### ================================
file_names = frame['date'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './date/date.txt'
    
    try:
    
        # Writing on file
        contents = frame['date'].iloc[key]
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"a")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  30
1  :  30
2  :  30
3  :  30
4  :  30
5  :  30
6  :  30
7  :  30
8  :  30
9  :  30
10  :  30
11  :  30
12  :  30
13  :  30
14  :  30
15  :  30
16  :  30
17  :  30
18  :  30
19  :  30
20  :  30
21  :  30
22  :  30
23  :  30
24  :  30
25  :  30
26  :  30
27  :  30
28  :  30
29  :  30
30  :  30
31  :  30
32  :  30
33  :  30
34  :  30
35  :  30
36  :  30
37  :  30
38  :  30
39  :  30
40  :  30
41  :  30
42  :  30
43  :  30
44  :  30
45  :  30
46  :  30
47  :  30
48  :  30
49  :  30
50  :  30
51  :  30
52  :  30
53  :  30
54  :  30
55  :  30
56  :  30
57  :  30
58  :  30
59  :  30
60  :  30
61  :  30
62  :  30
63  :  30
64  :  30
65  :  30
66  :  30
67  :  30
68  :  30
69  :  30
70  :  30
71  :  30
72  :  30
73  :  30
74  :  30
75  :  30
76  :  30
77  :  30
78  :  30
79  :  30
80  :  30
81  :  30
82  :  30
83  :  30
84  :  30
85  :  30
86  :  30
87  :  30
88  :  30
89  :  30
90  :  30
91  :  30
92  :  30
93  :  30
94  :  30
95  :  30
96  :  30
97  :  30
98  :  30
99  :  30
100  :  30

1030  :  30
1031  :  30
1032  :  30
1033  :  30
1034  :  30
1035  :  30
1036  :  30
1037  :  30
1038  :  30
1039  :  30
1040  :  30
1041  :  30
1042  :  30
1043  :  30
1044  :  30
1045  :  30
1046  :  30
1047  :  30
1048  :  30
1049  :  30
1050  :  30
1051  :  30
1052  :  30
1053  :  30
1054  :  30
1055  :  30
1056  :  30
1057  :  30
1058  :  30
1059  :  30
1060  :  30
1061  :  30
1062  :  30
1063  :  30
1064  :  30
1065  :  30
1066  :  30
1067  :  30
1068  :  30
1069  :  30
1070  :  30
1071  :  30
1072  :  30
1073  :  30
1074  :  30
1075  :  30
1076  :  30
1077  :  30
1078  :  30
1079  :  30
1080  :  30
1081  :  30
1082  :  30
1083  :  30
1084  :  30
1085  :  30
1086  :  30
1087  :  30
1088  :  30
1089  :  30
1090  :  30
1091  :  30
1092  :  30
1093  :  30
1094  :  30
1095  :  30
1096  :  30
1097  :  30
1098  :  30
1099  :  30
1100  :  30
1101  :  30
1102  :  30
1103  :  30
1104  :  30
1105  :  30
1106  :  30
1107  :  30
1108  :  30
1109  :  30
1110  :  30
1111  :  30
1112  :  30
1113

1715  :  30
1716  :  30
1717  :  30
1718  :  30
1719  :  30
1720  :  30
1721  :  30
1722  :  30
1723  :  30
1724  :  30
1725  :  30
1726  :  30
1727  :  30
1728  :  30
1729  :  30
1730  :  30
1731  :  30
1732  :  30
1733  :  30
1734  :  30
1735  :  30
1736  :  30
1737  :  30
1738  :  30
1739  :  30
1740  :  30
1741  :  30
1742  :  30
1743  :  30
1744  :  30
1745  :  30
1746  :  30
1747  :  30
1748  :  30
1749  :  30
1750  :  30
1751  :  30
1752  :  30
1753  :  30
1754  :  30
1755  :  30
1756  :  30
1757  :  30
1758  :  30
1759  :  30
1760  :  30
1761  :  30
1762  :  30
1763  :  30
1764  :  30
1765  :  30
1766  :  30
1767  :  30
1768  :  30
1769  :  30
1770  :  30
1771  :  30
1772  :  30
1773  :  30
1774  :  30
1775  :  30
1776  :  30
1777  :  30
1778  :  30
1779  :  30
1780  :  30
1781  :  30
1782  :  30
1783  :  30
1784  :  30
1785  :  30
1786  :  30
1787  :  30
1788  :  30
1789  :  30
1790  :  30
1791  :  30
1792  :  30
1793  :  30
1794  :  30
1795  :  30
1796  :  30
1797  :  30
1798

2446  :  30
2447  :  30
2448  :  30
2449  :  30
2450  :  30
2451  :  30
2452  :  30
2453  :  30
2454  :  30
2455  :  30
2456  :  30
2457  :  30
2458  :  30
2459  :  30
2460  :  30
2461  :  30
2462  :  30
2463  :  30
2464  :  30
2465  :  30
2466  :  30
2467  :  30
2468  :  30
2469  :  30
2470  :  30
2471  :  30
2472  :  30
2473  :  30
2474  :  30
2475  :  30
2476  :  30
2477  :  30
2478  :  30
2479  :  30
2480  :  30
2481  :  30
2482  :  30
2483  :  30
2484  :  30
2485  :  30
2486  :  30
2487  :  30
2488  :  30
2489  :  30
2490  :  30
2491  :  30
2492  :  30
2493  :  30
2494  :  30
2495  :  30
2496  :  30
2497  :  30
2498  :  30
2499  :  30
2500  :  30
2501  :  30
2502  :  30
2503  :  30
2504  :  30
2505  :  30
2506  :  30
2507  :  30
2508  :  30
2509  :  30
2510  :  30
2511  :  30
2512  :  30
2513  :  30
2514  :  30
2515  :  30
2516  :  30
2517  :  30
2518  :  30
2519  :  30
2520  :  30
2521  :  30
2522  :  30
2523  :  30
2524  :  30
2525  :  30
2526  :  30
2527  :  30
2528  :  30
2529

3206  :  30
3207  :  30
3208  :  30
3209  :  30
3210  :  30
3211  :  30
3212  :  30
3213  :  30
3214  :  30
3215  :  30
3216  :  30
3217  :  30
3218  :  30
3219  :  30
3220  :  30
3221  :  30
3222  :  30
3223  :  30
3224  :  30
3225  :  30
3226  :  30
3227  :  30
3228  :  30
3229  :  30
3230  :  30
3231  :  30
3232  :  30
3233  :  30
3234  :  30
3235  :  30
3236  :  30
3237  :  30
3238  :  30
3239  :  30
3240  :  30
3241  :  30
3242  :  30
3243  :  30
3244  :  30
3245  :  30
3246  :  30
3247  :  30
3248  :  30
3249  :  30
3250  :  30
3251  :  30
3252  :  30
3253  :  30
3254  :  30
3255  :  30
3256  :  30
3257  :  30
3258  :  30
3259  :  30
3260  :  30
3261  :  30
3262  :  30
3263  :  30
3264  :  30
3265  :  30
3266  :  30
3267  :  30
3268  :  30
3269  :  30
3270  :  30
3271  :  30
3272  :  30
3273  :  30
3274  :  30
3275  :  30
3276  :  30
3277  :  30
3278  :  30
3279  :  30
3280  :  30
3281  :  30
3282  :  30
3283  :  30
3284  :  30
3285  :  30
3286  :  30
3287  :  30
3288  :  30
3289

3941  :  30
3942  :  30
3943  :  30
3944  :  30
3945  :  30
3946  :  30
3947  :  30
3948  :  30
3949  :  30
3950  :  30
3951  :  30
3952  :  30
3953  :  30
3954  :  30
3955  :  30
3956  :  30
3957  :  30
3958  :  30
3959  :  30
3960  :  30
3961  :  30
3962  :  30
3963  :  30
3964  :  30
3965  :  30
3966  :  30
3967  :  30
3968  :  30
3969  :  30
3970  :  30
3971  :  30
3972  :  30
3973  :  30
3974  :  30
3975  :  30
3976  :  30
3977  :  30
3978  :  30
3979  :  30
3980  :  30
3981  :  30
3982  :  30
3983  :  30
3984  :  30
3985  :  30
3986  :  30
3987  :  30
3988  :  30
3989  :  30
3990  :  30
3991  :  30
3992  :  30
3993  :  30
3994  :  30
3995  :  30
3996  :  30
3997  :  30
3998  :  30
3999  :  30
4000  :  30
4001  :  30
4002  :  30
4003  :  30
4004  :  30
4005  :  30
4006  :  30
4007  :  30
4008  :  30
4009  :  30
4010  :  30
4011  :  30
4012  :  30
4013  :  30
4014  :  30
4015  :  30
4016  :  30
4017  :  30
4018  :  30
4019  :  30
4020  :  30
4021  :  30
4022  :  30
4023  :  30
4024

4702  :  30
4703  :  30
4704  :  30
4705  :  30
4706  :  30
4707  :  30
4708  :  30
4709  :  30
4710  :  30
4711  :  30
4712  :  30
4713  :  30
4714  :  30
4715  :  30
4716  :  30
4717  :  30
4718  :  30
4719  :  30
4720  :  30
4721  :  30
4722  :  30
4723  :  30
4724  :  30
4725  :  30
4726  :  30
4727  :  30
4728  :  30
4729  :  30
4730  :  30
4731  :  30
4732  :  30
4733  :  30
4734  :  30
4735  :  30
4736  :  30
4737  :  30
4738  :  30
4739  :  30
4740  :  30
4741  :  30
4742  :  30
4743  :  30
4744  :  30
4745  :  30
4746  :  30
4747  :  30
4748  :  30
4749  :  30
4750  :  30
4751  :  30
4752  :  30
4753  :  30
4754  :  30
4755  :  30
4756  :  30
4757  :  30
4758  :  30
4759  :  30
4760  :  30
4761  :  30
4762  :  30
4763  :  30
4764  :  30
4765  :  30
4766  :  30
4767  :  30
4768  :  30
4769  :  30
4770  :  30
4771  :  30
4772  :  30
4773  :  30
4774  :  30
4775  :  30
4776  :  30
4777  :  30
4778  :  30
4779  :  30
4780  :  30
4781  :  30
4782  :  30
4783  :  30
4784  :  30
4785

5450  :  30
5451  :  30
5452  :  30
5453  :  30
5454  :  30
5455  :  30
5456  :  30
5457  :  30
5458  :  30
5459  :  30
5460  :  30
5461  :  30
5462  :  30
5463  :  30
5464  :  30
5465  :  30
5466  :  30
5467  :  30
5468  :  30
5469  :  30
5470  :  30
5471  :  30
5472  :  30
5473  :  30
5474  :  30
5475  :  30
5476  :  30
5477  :  30
5478  :  30
5479  :  30
5480  :  30
5481  :  30
5482  :  30
5483  :  30
5484  :  30
5485  :  30
5486  :  30
5487  :  30
5488  :  30
5489  :  30
5490  :  30
5491  :  30
5492  :  30
5493  :  30
5494  :  30
5495  :  30
5496  :  30
5497  :  30
5498  :  30
5499  :  30
5500  :  30
5501  :  30
5502  :  30
5503  :  30
5504  :  30
5505  :  30
5506  :  30
5507  :  30
5508  :  30
5509  :  30
5510  :  30
5511  :  30
5512  :  30
5513  :  30
5514  :  30
5515  :  30
5516  :  30
5517  :  30
5518  :  30
5519  :  30
5520  :  30
5521  :  30
5522  :  30
5523  :  30
5524  :  30
5525  :  30
5526  :  30
5527  :  30
5528  :  30
5529  :  30
5530  :  30
5531  :  30
5532  :  30
5533